# Teil 2: Absichern des Model Serving mit Syft Keras

Nachdem das Model mit normalem Keras trainiert wurde, sind Sie nun bereit für vertrauliche Vorhersagen. Diese können mit Syft Keras umgesetzt werden.

Um das Model sicher anbieten zu können, werden drei TFEWorker (Server) benötigt. Weil TF Encrypted auf [Multi-Party Computation (MPC)](https://en.wikipedia.org/wiki/Secure_multi-party_computation) genannten Verschlüsselungen beruht, ist dies nötig. Die Idee dahinter ist, die Gewichte des Models additiv aufzuteilen in Anteile, welche anschließend auf die unterschiedlichen Server verteilt werden. Die wichtigste Eigenschaft dabei ist, dass der einzelne Server keinerlei Aussagen treffen kann über die Daten oder die Gewichte des Models.

Es wird ein Syft Keras Model wie im vorherigen Notebook definiert. Bevor dies jedoch geschieht, wird Keras noch angebunden mit `hook = sy.KerasHook(tf.keras)`. Dadurch werden drei wichtige neue Methoden zur Keras Sequential Klasse hinzugefügt:
 - `share`: sichert das Model mit additivem Aufteilen ab; standardmäßig wird das SecureNN Protokoll von TF Encrypted verwendet, um die Anteile auf die drei TFEWorker zu verteilen. Der wichtigste Punkt ist der, dass dadurch Vorhersagen auf verschlüsselten Daten ermöglicht werden.
 - `serve`: diese Funktion startet eine Queue, sodass die TFEWorker Vorhersage-Anfragen an das abgesicherte Model von externen Klienten akzeptieren können.
 - `shutdown_workers`: sobald alle vertraulichen Vorhersagen getätigt sind, kann hiermit das Model gestoppt werden. Auchkönnen Anleitungen bereitgestellt werden fürs manuelle Abschalten, sollte dies gewünscht sein.

Wenn Sie mehr über MPC erfahren wollen, können sie dies auf diesem exzellenten [Blog](https://mortendahl.github.io/2017/04/17/private-deep-learning-with-mpc/).

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

import syft as sy
hook = sy.KerasHook(tf.keras)

## Model

Wie zu erkennen ist, wurde beinahe eine exakte Kopie des vorherigen Models erstellt. Einzig und allein wurde noch `batch_input_shape` mit angegeben. Dies ermöglicht TF Encrypted die sichere Berechnung besser zu optimieren, da die Tensorgröße bekannt ist. Für diese MNIST Demonstration werden die Eingabe-Daten in der Form (1, 28, 28, 1) verwendet.  
Auch werden die Logits anstelle des Softmaxes zurückgegeben, dies reduziert die Komplexizität für das MPC deutlich  und wird für die Vorhersagen nicht benötigt.

In [ ]:
num_classes = 10
input_shape = (1, 28, 28, 1)

In [ ]:
model = Sequential()

model.add(Conv2D(10, (3, 3), batch_input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, name="logit"))

### Laden der Vortrainierten Gewichte

Mit `load_weights` werden die vorher gespeicherten Gewichte des trainierten Models einfach wieder verwendet.

In [ ]:
pre_trained_weights = 'short-conv-mnist.h5'
model.load_weights(pre_trained_weights)

### Starten der Helfer

Nun werden die TFEWorker (`alice`, `bob`, and `carol`) erstellt um mit TF Encrypted vertrauliche Vorhersagen anbieten zu können. Für jeden TFEWorker muss ein Host angegeben werden. Anschließend können die Helfer in einem Cluster gebündelt werden.

Diese Helfer starten einen [TensorFlow Server](https://www.tensorflow.org/api_docs/python/tf/distribute/Server), welchen Sie selbst manuell (`AUTO = False`) oder automatisiert (`AUTO = True`) verwalten können. Wenn Sie manuelles Verwalten wählen, werden Sie nach `cluster.start()` angeleitet einen Befehl im Terminal für jeden Helfer auszuführen. Sollten alle Helfer auf einem einzigen Gerät (z. B. `localhost`) sein, kann Syft das automatische Verwalten der TensorFlow Server übernehme.

In [ ]:
AUTO = True

alice = sy.TFEWorker(host='localhost:4000', auto_managed=AUTO)
bob = sy.TFEWorker(host='localhost:4001', auto_managed=AUTO)
carol = sy.TFEWorker(host='localhost:4002', auto_managed=AUTO)

cluster = sy.TFECluster(alice, bob, carol)
cluster.start()

### Absichern des Models durch aufteilen der Gewichte

Dank `sy.KerasHook(tf.keras)` lässt sich die `share` Methode aufrufen, um das Model in ein TF Encrypted Keras Model umzuwandeln. 

Wenn Sie sich für eine manuelle Verwaltung der Server entschieden haben, wird dieser Prozess nicht abgeschlossen werden können, bis nicht alle Helfer gestartet sind. Anzumerken ist, dass Ihre Firewall möglicherweise nachfragt, ob Python eingehende Verbindungen eingehen darf.

In [ ]:
model.share(cluster)

### Anbieten des Models

Perfekt! Mit dem Aufruf von `model.serve()` ist das Model bereit vertrauliche Vorhersagen anzubieten. Mit `num_requests` kann ein Limit für die Anzahl an Vorhersagen gesetzt werden; wird dies nicht angegeben, so bleibt das Model in Bereitschaft bis es unterbrochen wird.

In [ ]:
model.serve(num_requests=3)

Sie sind bereit zum Notebook **Part 13c** zu wechseln und dort einige vertrauliche Vorhersage-Anfragen zu starten.

### Aufräumen!

Sobald das Limit oben erreicht ist, wird das Model nicht mehr bereit stehen für Anfragen, doch bleibt es weiterhin geteilt auf allen drei Helfern. Diese Helfer können mit der unteren Zelle abgeschaltet werden. 

**Glückwunsch** zum Abschluss von Teil 13b: 

Once your request limit above, the model will no longer be available for serving requests, but it's still secret shared between the three workers above. You can kill the workers by executing the cell below.

**Congratulations** on finishing Part 13b: Absichern des Model Serving mit Syft Keras

In [ ]:
model.stop()
cluster.stop()

if not AUTO:
    process_ids = !ps aux | grep '[p]ython -m tf_encrypted.player --config' | awk '{print $2}'
    for process_id in process_ids:
        !kill {process_id}
        print("Process ID {id} has been killed.".format(id=process_id))